# Falsche Files finden und code anpassen

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import requests, zipfile, io #zum abspeichern von PDFs
from tqdm import tqdm
import PyPDF2
import re
from os import listdir
from os.path import isfile, join

### Die einzelnen Geschäfte aufrufen und Infos rausholen

In [33]:
#nun muss ich den Code ändern, sodass ich nicht mehr online zugreiffe sondern über meinen Ordner.
#hier nun die Liste der Zugriffe.
geschaefts_liste=[]#####ACHTUNG ich mache den RANGE kleiner, um weniger GEschäfte zu erhalten.

for seite_g in range(0,1220): #Range für alle Geschäfte wäre 1219!
    gesch= "KRhtml/Geschaeft"+str(seite_g)+".html"
    geschaefts_liste.append(gesch)

In [34]:
len(geschaefts_liste)

1220

In [35]:
geschaefts_liste[-10:]

['KRhtml/Geschaeft1210.html',
 'KRhtml/Geschaeft1211.html',
 'KRhtml/Geschaeft1212.html',
 'KRhtml/Geschaeft1213.html',
 'KRhtml/Geschaeft1214.html',
 'KRhtml/Geschaeft1215.html',
 'KRhtml/Geschaeft1216.html',
 'KRhtml/Geschaeft1217.html',
 'KRhtml/Geschaeft1218.html',
 'KRhtml/Geschaeft1219.html']

# Nun den Zugang zu den PDFs

### Ich muss noch rausfinden, bei welchen Geschäften, wie ausgelesen werden muss.

In [36]:
# ich will hier mal schauen, wieviele  Geschäfte mit -1 funktionieren und wiviele nicht.
truelist = []
falselist = []
for geschaeft in geschaefts_liste:
    file = open(geschaeft, 'r')
    text = file.read()
    soup_g = BeautifulSoup(text, 'html.parser')
    tr_g_list=soup_g.find_all("tr")
    td_g_list=tr_g_list[-1].find_all("td")
    if td_g_list[1].find_all("a")[0].text== "1":
        truelist.append(geschaeft)
    else:
        falselist.append(geschaeft)
        

In [37]:
len(truelist)

1061

In [1]:
len(falselist)

NameError: name 'falselist' is not defined

In [39]:
# und nun jene von den falschen, die mit -2 funktionieren
# MANN geht nicht. Am Ende hat es eine leere Klammer, die bringe ich auch nicht weg, wenn ich das letzte Geschäft entferne
# Es muss eines in der Mitte sein. Wie kann ich den Fehelr umgehen
# Bin mit der Listenposition die GEschäfte duerchgegangen. ab 127 ist Problem.
truelist_f = []
falselist_f = []
for geschaeft_f in falselist[:126]:
    file = open(geschaeft_f, 'r')
    text = file.read()
    soup_g = BeautifulSoup(text, 'html.parser')
    tr_f_list=soup_g.find_all("tr")
    td_f_list=tr_f_list[-2].find_all("td")
   
    if  td_f_list[1].find_all("a")[0].text == "1":
        truelist_f.append(geschaeft_f)
    else:
        falselist_f.append(geschaeft_f) 

In [53]:
# Die problematischen GEschäfte haben keinen Vorstoss 1 mehr drin. 
# Diese muss ich mir einzeln runterziehen, da sie auch anders ausgelesen werden müssen.  
ohne_vor_list_2=[]
falselist_vor=[]
for geschaeft_f in falselist[127:]:
    file = open(geschaeft_f, 'r')
    text = file.read()
    soup_g = BeautifulSoup(text, 'html.parser')
    tr_f_list=soup_g.find_all("tr")
    td_f_list=tr_f_list[-1].find_all("td")
   
    if  td_f_list[1].find_all("a")[0].text == "2":
        ohne_vor_list_2.append(geschaeft_f)
    else:
        falselist_vor.append(geschaeft_f) 

In [56]:
ohne_vor_list_2 #Alle Vorstösse, bei welchen der zweite Teil rausgezogen werden müssen

['KRhtml/Geschaeft1184.html',
 'KRhtml/Geschaeft1186.html',
 'KRhtml/Geschaeft1187.html',
 'KRhtml/Geschaeft1188.html',
 'KRhtml/Geschaeft1189.html',
 'KRhtml/Geschaeft1190.html',
 'KRhtml/Geschaeft1191.html',
 'KRhtml/Geschaeft1193.html',
 'KRhtml/Geschaeft1194.html',
 'KRhtml/Geschaeft1195.html',
 'KRhtml/Geschaeft1196.html',
 'KRhtml/Geschaeft1197.html',
 'KRhtml/Geschaeft1198.html',
 'KRhtml/Geschaeft1199.html',
 'KRhtml/Geschaeft1202.html',
 'KRhtml/Geschaeft1203.html',
 'KRhtml/Geschaeft1204.html',
 'KRhtml/Geschaeft1205.html',
 'KRhtml/Geschaeft1206.html',
 'KRhtml/Geschaeft1207.html',
 'KRhtml/Geschaeft1208.html',
 'KRhtml/Geschaeft1209.html',
 'KRhtml/Geschaeft1210.html',
 'KRhtml/Geschaeft1211.html',
 'KRhtml/Geschaeft1212.html',
 'KRhtml/Geschaeft1213.html',
 'KRhtml/Geschaeft1214.html',
 'KRhtml/Geschaeft1217.html']

In [55]:
falselist_vor # Hier alle Vorstösse, die mit == "3" ausgelesen werden müssen

['KRhtml/Geschaeft1215.html',
 'KRhtml/Geschaeft1216.html',
 'KRhtml/Geschaeft1218.html',
 'KRhtml/Geschaeft1219.html']

# Nun die Erkenntniss, dass nur die auszulesende Zahl nicht stimmt einfügen

In [58]:
#hier  habe ich mich entschieden, auf den Zugriff der Namen zu verzichten, und dies einheitlich über die PDFs zu lösen.
#Nun seite für Seite abrufen und in einem neuen Minidict unterbringen
#ACHTUNG überprüfen: es bringt zumindest für einige die falschen Daten raus. und dann gleich mehrfach. 
#(Geschäft: 2129 8fach. Nur das richtige Datum ist nicht drinn.)
eingang_list=[]


for geschaeft in geschaefts_liste:
    file = open(geschaeft, 'r')
    text = file.read()
    soup_g = BeautifulSoup(text, 'html.parser')
    tr_g_list=soup_g.find_all("tr")
    td_g_list=tr_g_list[-1].find_all("td")#hier gehe ich zum Dokument.
    gesch_nr = tr_g_list[-1].find_all("td")[0].text
    einger_am = tr_g_list[1].find_all("td")[0].text   
    ul_list = tr_g_list[2].find_all("ul")
    li_list = ul_list[0].find_all("li")
   
     #Da es zum Teil Referenzen hat, muss ich varieren.
    
    if td_g_list[-1].find_all("a")[0].text== "1": #Ich brauche immer das Dokument 1. 
        url_v_pdf = td_g_list[-1].find("a")["href"]
    elif td_g_list[-1].find_all("a")[0].text== "2":
        url_v_pdf = td_g_list[-1].find("a")["href"]   
    else:  
        td_g_list[-1].find_all("a")[0].text== "3"
        url_v_pdf = td_g_list[-1].find("a")["href"]
    
   

    #ACHTUNG: dieses if-Konstrukt funktioniert nicht - doch if/else? Es kommt immer dasselbe resultat raus.        
    

        
    
    minidict_g={"VorlageNR":gesch_nr, "Einreichedatum":einger_am, "Link Vorstoss-PDF":url_v_pdf}
    for key, value in minidict_g.items():
        if value == '':
            minidict_g[key] = 'NaN'  #hier schaue ich noch, dass ich die Leeren Zeilen mit NAN ersetzten kann.
        else:
            minidict_g[key] = value
            
    eingang_list.append(minidict_g)

In [59]:
df_g=pd.DataFrame(eingang_list)

In [60]:
df_g

,VorlageNR,Einreichedatum,Link Vorstoss-PDF
0,3020,11.10.2019,/dokumente/8316/3020-1-16168_Racial-Profiling.pdf
1,3019,11.10.2019,/dokumente/8317/3019-1-16167_Chancengleichheit...
2,3018,07.10.2019,/dokumente/8310/3018-1-16166_Praktikum.pdf
3,3017,06.10.2019,/dokumente/8311/3017-1-16165_Frauenwahl-stimmr...
4,3014,26.09.2019,/dokumente/8312/3014-1-16159_Geschwindigkeitsk...
...,...,...,...
1215,762,NaN,/dokumente/4232/pdoc_1047_1.pdf
1216,666,NaN,/dokumente/4215/pdoc_55_1.pdf
1217,304,NaN,/dokumente/4179/pdoc_50_1.pdf
1218,2635,07.08.1995,/gast/geschaefte/1620


In [ ]:
df_g["Link Vorstoss-PDF"]

# PDFs der Vorstösse runterladen

In [ ]:
for link in tqdm(df_g["Link Vorstoss-PDF"]):
    r = requests.get("https://kr-geschaefte.zug.ch"+link, stream =True) #Das Stream braucht es, um mit dem Zip umgehen zu koennen
    name = link.split("/")[-1] #Der Computer kann mit "/" in einem Namen nicht umgehen, deshalb nehme ich die hier raus, und nehme nur den letzten Teil des NAmens.
    with open("KRGeschPDF/"+name, "wb") as f:
        f.write(r.content)

# PDFs sind auf dem Rechner, nun lesbar machen und !Regex Baby!

In [ ]:
#nun will ich aus jedem PDF zum Einen die VorlagenNR rausziehen sowie Namen und Datum hier die Definition
# ! Danke dem |$ am Schluss gibt es keinen Index-Error, wenn er kein Resultat findet.
regex_vorlage= r"VORLAGE.NR\..(\d*)\.1|$" #hier ziehe ich die Vorlagennummer raus. 
regex_datum = r"VOM.(\d+\..\w*.\d{4})|$" # hier gehe ich an das DAtum
regex_name=r"\bVON(.*UND.*)" #Hier muss ich noch überlegen, wie anders. Problem: zum Teil sind Namen einzeln zum teil mehr
# Und irgendwie erkennt er die Zeilenende nicht.
regex_name1= r"\bVON.(\w*.\w*\b)"
regex_partei= r"DER.(\w*)"

In [ ]:
from os import listdir
from os.path import isfile, join


In [ ]:
#Lösung online gefunden um aus dem PDF-Ordner eine Liste mit den Links herzustellen.
pdf_files_list= [f for f in listdir("KRGeschPDF") if isfile(join("KRGeschPDF", f))]

In [ ]:
pdf_files_list

In [ ]:
# !!!!!! ACHTUNG: zuerst mit einzelen testen. Möglicherweise findet er die Files nicht. 
dat_pdf_list=[]
for dok in pdf_files_list:
    mypdf = open("KRGeschPDF/"+dok, mode='rb')
    pdf_document = PyPDF2.PdfFileReader(mypdf)
    first_page = pdf_document.getPage(0)
    front=first_page.extractText()
    dat_p_ein= re.findall(regex_datum, front, re.IGNORECASE)[0] #Ich greiffe auf das Datum zu
        # um zu verhindern, dass ich mehrer Daten und Vorlagen habe, sage ich mit [0] er soll das erste nehmen
    vorl_p_nr= re.findall(regex_vorlage, front, re.IGNORECASE)[0] # Ich greiffe auf die Vorlagennummer zu (zum Zusammenfügen der Dataframes)
    
    minidict_pdf_dat={"Einreichedatum": dat_p_ein, "VorlageNR":vorl_p_nr}
    
    dat_pdf_list.append(minidict_pdf_dat)

In [ ]:
df_p=pd.DataFrame(dat_pdf_list)

In [ ]:
df_p

# Das Datenformat in englisch umwandeln und lesbar machen.

In [ ]:
# Hmmmmm jetzt chumi nümm wiiter. Muss die Monate ins Englische umwandeln.
d={"JANUAR":"January", "FEBRUAR":"February", "MÄRZ":"March", "APRIL":"April", "MAI":"May","JUNI":"June","JULI":"Juli","AUGUST":"August","SEPTEMBER":"September","OKTOBER":"October","NOVEMBER":"November", "DEZEMBER":"December"}


# Zum Resultat hier mal das Dataframe der Hauptseite sowie der Unterseiten zusammenfügen.

In [ ]:
df_mit_dat=pd.merge(df_a, df_g, how="left", on="GeschäftsNR") #ich verbinde die beiden DataFrames

In [ ]:
#nun will ich noch das Datum zum Idex machen
df_mit_dat.set_index("Einreichedatum", inplace=True)

In [ ]:
df_mit_dat.dtypes

# Ein erstes Ergebnis = ab 2013 alle  Vorstösse
# ACHTUNG das stimmt ja so gar nicht, da bei mehreren Personen
# auch mehrere Einreichungen verzeichnet werden!!!

In [ ]:
#Nun will ich ein erstes Mal in die DAten schauen, zumindest ab 2013 - ob es ein Muster beim Einreichen gibt.
#DAzu krame ich jetzt alle Vorstösse raus

df_vo_all=df_mit_dat[{"Art des Geschäfts" : ["Motion","Interpellation","Postulat","Kleine Anfrage"] }]

In [ ]:
df_mit_dat["Art des Geschäfts"]["GeschäftsNR"]

In [ ]:
df_vo_all["2013":].resample("SMS").count().plot(figsize=(8,4))

# Regex Baby! 
### Ich versuche mit Regex in Pandas an die Namen zu kommen

In [ ]:
#nun versuche ich mit Regex an die einzelnen Namen zu kommen.
regex_n=r"\bvon(.*und.\w*.\w*\b)"
regex_n2= r"^(\w*.\w*\b)\,.(\w*.\w*\b)"

In [ ]:
df_n1=df_a['Geschäft'].str.extract(regex_n)

In [ ]:
df_n1